<a href="https://colab.research.google.com/github/lorenafc/MGI/blob/main/Maplapse_Complete.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#import libraries
import pandas as pd
import geopandas as gpd
import folium
from folium import Marker
import matplotlib.pyplot as plt


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# upload the file from your computer
from google.colab import files
uploaded=files.upload()

Saving merged_additional_authors_encoded.xlsx to merged_additional_authors_encoded.xlsx


#CHANGE THE FILE WHEN YOU CLEAN IT FOR THE MAPLAPSE, AND ADD THE COUNT

In [4]:
# Create a dataframe, replace the file_name with your excel file name
file_name = 'merged_additional_authors_encoded.xlsx'

authors_small_cities = pd.read_excel(file_name, engine='openpyxl')

In [6]:
authors_small_cities_copy=authors_small_cities.copy()


In [8]:
authors_small_cities_copy.shape

(80802, 15)

In [10]:
authors_small_cities_copy.rename(columns={'Unnamed: 0': 'indexauthor'}, inplace=True)
authors_small_cities_copy.head()

,indexauthor,starturl,bitrhyear,deathyear,nameandbirthdeathyear,georeferenceurl,borncity,deathcity,activecity,latitude_born,longitude_born,latitude_death,longitude_death,latitude_active,longitude_active
0,0,http://viaf.org/viaf/81048146,1570,1634.0,"Carneiro, Tomé Tavares 1570-1634",viafprocessedfile,,NaN,Rio Tinto,NaN,NaN,NaN,NaN,-6.803828,-35.077569
1,1,http://viaf.org/viaf/180581759,NaN,1775.0,"Kolíček, Caesarij, -1775",viafprocessedfile,,Cieszin,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,http://viaf.org/viaf/100161519,NaN,1570.0,"Hertel, Jakob 15..?-1570",http://catalogue.bnf.fr/ark:/12148/cb15011774s,(Chur,NaN,NaN,46.854747,9.526490,NaN,NaN,NaN,NaN
3,3,http://viaf.org/viaf/169350346,1699,1770.0,"Ferro Caaveyro, Lucas Antonio ca. 1699-1770",www.wikidata.org/wiki/Q3393623#sitelinks-wikip...,A Capela,NaN,NaN,43.442068,-8.043400,NaN,NaN,NaN,NaN
4,4,http://viaf.org/viaf/67541728,1502,1600.0,"Aparicio, Sebastián de 1502-1600 Santo",www.wikidata.org/wiki/Q2486114#sitelinks-wikip...,A Gudiña,Puebla City,NaN,42.060548,-7.140352,19.04372,-98.198149,NaN,NaN


In [11]:
# Count the occurrences of each city in the 'activecity' column
city_counts = authors_small_cities_copy['activecity'].value_counts()

# Map the counts to the 'activecity' column
authors_small_cities_copy['activecitycount'] = authors_small_cities_copy['activecity'].map(city_counts)

print(authors_small_cities_copy.head(10))

   indexauthor                        starturl bitrhyear  deathyear  \
0            0   http://viaf.org/viaf/81048146      1570     1634.0   
1            1  http://viaf.org/viaf/180581759       NaN     1775.0   
2            2  http://viaf.org/viaf/100161519       NaN     1570.0   
3            3  http://viaf.org/viaf/169350346      1699     1770.0   
4            4   http://viaf.org/viaf/67541728      1502     1600.0   
5            5  http://viaf.org/viaf/121452165      1320     1394.0   
6            6   http://viaf.org/viaf/62383957      1538     1592.0   
7            7   http://viaf.org/viaf/67541507      1530     1596.0   
8            8   http://viaf.org/viaf/78669554      1640     1705.0   
9            9   http://viaf.org/viaf/79445202      1731     1797.0   

                         nameandbirthdeathyear  \
0             Carneiro, Tomé Tavares 1570-1634   
1                     Kolíček, Caesarij, -1775   
2                     Hertel, Jakob 15..?-1570   
3  Ferro Caaveyro,

In [12]:
authors_small_cities_copy.shape

(80802, 16)

In [20]:
# THIS ADJUSTMENTS FOR THE MAPLAPSE LIBRARY


# Create the 'countcitygroupby' column
authors_small_cities_copy['countcitygroupby'] = authors_small_cities_copy.groupby('activecity')['activecity'].transform('count') # Remove the immediate type casting

# Create a new DataFrame with only the required columns
required_columns = ["bitrhyear", 'deathyear', 'activecity', 'latitude_active', 'longitude_active', 'countcitygroupby']
authors_maplapse_active = authors_small_cities_copy[required_columns].copy()

# Handle or remove non-finite values in 'countcity' if necessary
# Example: Replace non-finite values with 0
authors_maplapse_active['countcityactivegroupby'] = authors_maplapse['countcitygroupby'].fillna(0).astype(int) # Fill NaN values and then convert to int

# Drop rows where 'latitude_active' column has NaN values
authors_maplapse_active.dropna(subset=['latitude_active'], inplace=True)

# Now 'authors_maplapse' contains only the columns you need
print(authors_maplapse_active.head())

   bitrhyear  deathyear  activecity  latitude_active  longitude_active  \
0       1570     1634.0   Rio Tinto        -6.803828        -35.077569   
10      1706     1761.0      Prešov        49.000007         21.239212   
17      1716     1776.0  Königgrätz        50.209211         15.832751   
19      1618     1678.0     Kolberg        54.176068         15.576554   
30       NaN     1743.0        Żary        51.624438         15.148725   

    countcitygroupby  countcityactivegroupby  
0                1.0                     1.0  
10               2.0                     2.0  
17               3.0                     3.0  
19              28.0                    28.0  
30               2.0                     2.0  


In [21]:

# Filter the DataFrame for the specified year range
authors1000_800to1800 = authors_maplapse_actived[
    (authors_maplapse_actived['deathyear'] >= 800) & (authors_maplapse_actived['deathyear'] <= 1800)
].copy()

# Display the first few rows of the new filtered DataFrame
print(authors_maplapse_active.head())


NameError: name 'authors_maplapse_actived' is not defined

In [ ]:

# selected_columns = [
#     'nameandbirthdeathyear','bitrhyear', 'deathyear',
#     'borncity', 'deathcity', 'activecity', 'activecity_latitude',
#     'activecity_longitude', 'latitude_active', 'longitude_active',
#     'latitude_born', 'longitude_born', 'latitude_death', 'longitude_death'
# ]

# # Create a new DataFrame with only the selected columns
# authors1000_cleaned = authors_small_cities_copy[selected_columns].copy()

# # Display the first few rows of the new DataFrame
# print(authors1000_cleaned.head())


In [ ]:
import pandas as pd

# Assuming 'authors1000_800to1800' is your existing DataFrame
# Create the 'countcity' column
authors1000_800to1800['countcity'] = authors1000_800to1800.groupby('activecity')['activecity'].transform('count') # Remove the immediate type casting

# Create a new DataFrame with only the required columns
required_columns = ["bitrhyear", 'deathyear', 'activecity', 'latitude_active', 'longitude_active', 'countcity']
authors_maplapse = authors1000_800to1800[required_columns].copy()

# Handle or remove non-finite values in 'countcity' if necessary
# Example: Replace non-finite values with 0
authors_maplapse['countcity'] = authors_maplapse['countcity'].fillna(0).astype(int) # Fill NaN values and then convert to int

# Drop rows where 'activecity' column has NaN values
authors_maplapse.dropna(subset=['activecity'], inplace=True)

# Now 'authors_maplapse' contains only the columns you need
print(authors_maplapse.head(20))

In [19]:
authors_maplapse_active.dtypes

bitrhyear                  object
deathyear                 float64
activecity                 object
latitude_active           float64
longitude_active          float64
countcitygroupby          float64
countcityactivegroupby    float64
dtype: object

In [ ]:
# export the result as excel file
file_name = 'authors1000_800to1800_maplapse.csv'
authors_maplapse.to_csv(file_name, index=False)

In [ ]:
# Specify the path in Google Drive
file_path = '/content/drive/My Drive/driv_authors1000_800to1800_maplapse.csv'

# Save the DataFrame to an Excel file in Google Drive
authors_maplapse.to_csv(file_path, index=False, header=True)

In [ ]:
authors1000_800to1800.dtypes

In [ ]:
authors_small_cities_copy = authors_maplapse

In [ ]:
authors_small_cities_copy.shape

#HEATMAP (ALL RECORDS). Born - red, Active- Green, Death - Blue

In [33]:
import geopandas as gpd
from shapely.geometry import Point
import folium
from folium.plugins import HeatMap

# Function to safely create points, handling NaN values
def create_point(lat, lon):
    if pd.notnull(lat) and pd.notnull(lon):
        return Point(lon, lat)
    return None

# Apply the function to each pair of latitude-longitude columns
authors_small_cities_copy['geometry_born'] = authors_small_cities_copy.apply(lambda row: create_point(row['latitude_born'], row['longitude_born']), axis=1)
authors_small_cities_copy['geometry_active'] = authors_small_cities_copy.apply(lambda row: create_point(row['latitude_active'], row['longitude_active']), axis=1)
authors_small_cities_copy['geometry_death'] = authors_small_cities_copy.apply(lambda row: create_point(row['latitude_death'], row['longitude_death']), axis=1)

# Create lists of points for born, active, and death locations
locations_born = [(point.y, point.x) for point in authors_small_cities_copy['geometry_born'] if point is not None]
locations_active = [(point.y, point.x) for point in authors_small_cities_copy['geometry_active'] if point is not None]
locations_death = [(point.y, point.x) for point in authors_small_cities_copy['geometry_death'] if point is not None]

# Create a map centered on Prague (50.08781, 14.42046)
m = folium.Map(location=[50.08781, 14.42046], tiles='cartodbpositron', zoom_start=4, control_scale=True)

# Add heatmaps with different colors for each category
HeatMap(locations_born, gradient={0.2: 'red', 0.4: 'darkred', 0.6: 'maroon'}).add_to(m)
HeatMap(locations_active, gradient={0.2: 'green', 0.4: 'limegreen', 0.6: 'darkgreen'}).add_to(m)
HeatMap(locations_death, gradient={0.2: 'blue', 0.4: 'royalblue', 0.6: 'navy'}).add_to(m)

# Visualize the result
m


In [35]:
# Save the interactive map html format
outfp="heat_map_authors_total_zoom1.html"
# Exporte o mapa
m.save(outfp)

##MARKERS

In [32]:
import folium

# Create a map centered on Prague (50.08781, 14.42046)
m = folium.Map(location=[50.08781, 14.42046], tiles='cartodbpositron', zoom_start=4, control_scale=True)

# Function to add markers to the map
def add_markers(locations, color, category_name):
    for lat, lon in locations:
        folium.Marker(
            location=[lat, lon],
            icon=folium.Icon(color=color),
            popup=f'{category_name}: {lat}, {lon}'
        ).add_to(m)

# Add markers for born, active, and death locations
add_markers(locations_born, 'red', 'Born')
add_markers(locations_active, 'green', 'Active')
add_markers(locations_death, 'blue', 'Death')

# Visualize the result
m

# Save the interactive map html format
outfp="heat_map_authors_total_zoom1.html"
# Exporte o mapa
m.save(outfp)

## TIMELAPSE WITH YEARS FROM 800 TO 1800

In [ ]:
!pip install maplapse -q

In [ ]:
# Maplapse. source: https://maplapse.readthedocs.io/en/latest/
# https://medium.com/udacity/creating-map-animations-with-python-97e24040f17b

from maplapse import Animator


# Create an Animator object with the appropriate parameters
# Replace 'None' with the actual default value if the library provides a default world map
anim = Animator(shape="/content/World_Cities.shp",  # Assuming the library provides a default world map
                value= "/content/authors1000_800to1800_maplapse.csv",  # Pass the DataFrame directly
                data_column="countcity",
                time_column='deathyear',  # Column that contains the years
                map_type='proportional_circle',  # Use 'proportional_circle' for circle markers
                scale_factor=0.02,
                out_path='/path_to_output/maplapse800_1800.mp4',  # Path to save the animation
                shape_unique_column="CITY_NAME")

# Run the animation
anim.animate(duration=0.5)


In [ ]:
import folium
import pandas as pd

# Assuming 'df' is your DataFrame with the relevant columns
# Filter rows where 'deathyear' is between 800 and 1800
time_lapse_df = df[(df['deathyear'] >= 800) & (df['deathyear'] <= 1800)]

# Sort the DataFrame by 'deathyear' for chronological order
time_lapse_df = time_lapse_df.sort_values('deathyear')

# Create a base map
m = folium.Map(location=[50.08781, 14.42046], tiles='cartodbpositron', zoom_start=4, control_scale=True)

# Function to add markers to the map for a given year
def add_markers_for_year(year):
    # Filter the DataFrame for the given year
    year_df = time_lapse_df[time_lapse_df['deathyear'] == year]

    # Iterate over the rows in the DataFrame
    for index, row in year_df.iterrows():
        # Use 'active_latitude' and 'active_longitude' if available, otherwise 'latitude_death' and 'longitude_death'
        lat = row['latitude_active'] if pd.notnull(row['latitude_active']) else row['latitude_death']
        lon = row['longitude_active'] if pd.notnull(row['longitude_active']) else row['longitude_death']

        # Add a marker to the map if coordinates are available
        if pd.notnull(lat) and pd.notnull(lon):
            folium.Marker(
                location=[lat, lon],
                popup=f'{row["nameandbirthdeathyear"]} ({year})',
                icon=folium.Icon(color='cadetblue')
            ).add_to(m)

# Add markers for each year in the range
for year in range(800, 1801):
    add_markers_for_year(year)

# Save the map to an HTML file
m.save('chronological_markers.html')

#Display map
m

##below: the code worked for a sample of 100 geocoded locations. all the rows had a location, and it was made only for the born column (now we have 3 columns: born, death and active locations)

In [24]:
authors_small_cities_gdf = authors_small_cities_copy.copy()

In [25]:
# probably using this code for the final version, because there willl have no NaN handling issues
#source https://geopandas.org/en/stable/gallery/create_geopandas_from_pandas.html and chatgpt
#convert dataframe to a geodataframe

from shapely.geometry import Point

authors_small_cities_gdf = gpd.GeoDataFrame(authors_small_cities_copy, geometry=[Point(eval(x)) for x in authors_no_nan['bornlocation_geo']], crs="EPSG:4326")



NameError: name 'authors_no_nan' is not defined

In [ ]:
# handling with the NaN values to create a geodataframe (in this example, because it wont be necessary with the geocoded final version code)
# source https://geopandas.org/en/stable/gallery/create_geopandas_from_pandas.html and copilot

# import geopandas as gpd
# from shapely.geometry import Point

# # Function to safely evaluate points, handling NaN values
# def create_point(location):
#     try:
#         return Point(eval(location))
#     except (TypeError, ValueError):
#         return None

# # Apply the function to the 'bornlocation_geo' column
# authors_small_cities_copy['geometry'] = authors_small_cities_copy['bornlocation_geo'].apply(create_point)

# # Create the GeoDataFrame
# authors_small_cities_gdf = gpd.GeoDataFrame(authors_small_cities_copy, geometry='geometry', crs="EPSG:4326")


In [ ]:
authors_small_cities_gdf.head()

In [ ]:
#HEAT MAP - source PYGEO course

# Retrieve the x and y coordinates of each point
# authors_small_cities_gdf['x'] =authors_small_cities_gdf["geometry"].apply(lambda geom: geom.x)
# authors_small_cities_gdf['y'] =authors_small_cities_gdf["geometry"].apply(lambda geom: geom.y)


# Retrieve the x and y coordinates of each point. Fill the column with None if there is no geometry:
authors_small_cities_gdf['x'] = authors_small_cities_gdf["geometry"].apply(lambda geom: geom.x if geom is not None else None)
authors_small_cities_gdf['y'] = authors_small_cities_gdf["geometry"].apply(lambda geom: geom.y if geom is not None else None)


# Create a list of pairs (tuples) of x and y coordinates
locations=list(zip(authors_small_cities_gdf["x"], authors_small_cities_gdf["y"]))



In [ ]:
locations

In [ ]:
# heat map plugin
from folium.plugins import HeatMap

# Create a map, centered in Prague (50.08781, 14.42046)  CRS WGS84
m= folium.Map(location=[50.08781, 14.42046], tiles='cartodbpositron', zoom_start=4, control_scale=True)

# Add the heat map to the base map
HeatMap(locations).add_to(m)

# Visualize the result
m

In [ ]:
# Save the interactive map html format
outfp="heat_map_first100zoom1.html"
# Exporte o mapa
m.save(outfp)

In [ ]:
!pip install selenium -q

In [ ]:
# save the folium map as png
# source: https://stackoverflow.com/questions/53565979/export-a-folium-map-as-a-png
import selenium
import io
from PIL import Image

img_data = m._to_png(5)
img = Image.open(io.BytesIO(img_data))
img.save('authors_born_location_fist100recordszoom2.4.png')

INTERACTIVE MAP

In [ ]:
#chat gpt

# Create a new geometry column with swapped coordinates
authors_small_cities_gdf['geometry'] = authors_small_cities_gdf.apply(lambda row: Point(row['y'], row['x']), axis=1)

# Verify the new coordinates
print(authors_small_cities_gdf[['x', 'y', 'geometry']].head())

# Convert GeoDataFrame to GeoJSON
authors_born_geojson = authors_small_cities_gdf.to_json()

# Create a Folium map centered on Amsterdam
m = folium.Map(location=[52.3676, 4.9041], tiles="cartodbpositron", zoom_start=4.2, control_scale=True)

# Add GeoJSON layer to the map with markers
folium.GeoJson(
    authors_born_geojson,
    name="Born Locations",
    tooltip=folium.GeoJsonTooltip(fields=['bornlocation'], labels=True)
).add_to(m)

m


In [ ]:
# Save the interactive map html format
outfpmarkers="markers_map_first100born.html"
# Exporte o mapa
m.save(outfpmarkers)

In [ ]:
# save the folium map as png
# source: https://stackoverflow.com/questions/53565979/export-a-folium-map-as-a-png


img_data = m._to_png(5)
img = Image.open(io.BytesIO(img_data))
img.save('markers_authors_born_location_fist100recordszoom4.2_europe.png')

STATIC MAP

In [ ]:
import matplotlib.pyplot as plt
import geopandas as gpd

# Load the world map
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))

# Plot the world map
fig, ax = plt.subplots(figsize=(10, 10))
world.plot(ax=ax, color='lightgrey', edgecolor='black')

# Plot your data on top of the world map
# Make sure 'authors_small_cities_gdf' is a GeoDataFrame with valid geometries
authors_small_cities_gdf.plot(ax=ax, markersize=5, color='red')

# Add title
ax.set_title('Authors Birthplaces')

# Add north arrow
ax.annotate('N', xy=(0.1, 0.3), xytext=(0.1, 0.1),
            arrowprops=dict(facecolor='black', width=5, headwidth=12),
            ha='center', va='center', fontsize=20, xycoords=ax.transAxes)

# Create a legend
import matplotlib.lines as mlines
blue_line = mlines.Line2D([], [], color='red', marker='.', markersize=15, label='Birth Location')
ax.legend(handles=[blue_line], loc='lower right')

# Remove axis off
ax.set_axis_off()

# Save the figure
plt.savefig('authors_birth_locations_map.png', bbox_inches='tight')


MAP THE AUTHORS BIRTH AND DEATH LOCATIONS ALONG THE YEARS

In [23]:
import folium
from folium.plugins import MarkerCluster

# Initialize the map centered around Europe
m = folium.Map(location=[54.5260, 15.2551], zoom_start=4)

# Create a MarkerCluster to manage the markers
marker_cluster = MarkerCluster().add_to(m)

# Loop through each year in your dataset
for year in range(800, 1801):  # Adjust the range to include the year 1800
    # Filter the DataFrame for authors who were born on or after 800
    # and died on or before 1800
    year_data = authors_small_cities_gdf[
        (authors_small_cities_gdf['deathyear'] >= 800) &
        (authors_small_cities_gdf['deathyear'] <= year)
    ]

    # Add lines and markers for each author
    for _, row in year_data.iterrows():
        # Birth location marker
        folium.Marker(
            location=[row['bornlocation_geo'], row['bornlocation_geo']],
            popup=f"{row['nameandbirthdeathyear']} born",
            icon=folium.Icon(color='green')
        ).add_to(marker_cluster)

        # Death location marker
        folium.Marker(
            location=[row['deathlocation_geo'], row['deathlocation_geo']],
            popup=f"{row['nameandbirthdeathyear']} died",
            icon=folium.Icon(color='red')
        ).add_to(marker_cluster)

        # Line connecting birth and death locations
        folium.PolyLine(
            locations=[
                [row['bornlocation_geo'], row['bornlocation_geo']],
                [row['deathlocation_geo'], row['deathlocation_geo']]
            ],
            color='blue'
        ).add_to(m)

# Display the map
m.save('authors_map.html')


KeyError: 'bornlocation_geo'

In [ ]:
#source: copilot
import folium
from folium.plugins import MarkerCluster

# Initialize the map centered around Europe
m = folium.Map(location=[54.5260, 15.2551], zoom_start=4)

# Create a MarkerCluster to manage the markers
marker_cluster = MarkerCluster().add_to(m)

# Loop through each year in your dataset
for year in range(800, 1800):
    # Filter the DataFrame for the given year
    year_data = authors_small_cities_gdf[(authors_small_cities_gdf['birthyear'] <= year) & (authors_small_cities_gdf['deathyear'] >= year)]

    # Add lines and markers for each author
    for _, row in year_data.iterrows():
        # Birth location marker
        folium.Marker(
            location=[row['latitude_born'], row['longitude_born']],
            popup=f"{row['name']} born",
            icon=folium.Icon(color='green')
        ).add_to(marker_cluster)

        # Death location marker
        folium.Marker(
            location=[row['latitude_death'], row['longitude_death']],
            popup=f"{row['name']} died",
            icon=folium.Icon(color='red')
        ).add_to(marker_cluster)

        # Line connecting birth and death locations
        folium.PolyLine(
            locations=[
                [row['latitude_born'], row['longitude_born']],
                [row['latitude_death'], row['longitude_death']]
            ],
            color='blue'
        ).add_to(m)

# Display the map
m.save('authors_map.html')
